<a href="https://colab.research.google.com/github/Deok-Hun/Aiffel_Kun/blob/master/EX_6_%EB%A9%8B%EC%A7%84_%EC%9E%91%EC%82%AC%EA%B0%80_%EB%A7%8C%EB%93%A4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import glob  #glob 모듈의 glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환한다
import tensorflow

print(tensorflow.__version__)

2.8.2


In [3]:
import os, re

In [4]:
os.environ["HOME"]=""
print(os.getenv("HOME"))

In [5]:
import numpy as np
import tensorflow as tf

# 데이터 읽어오기

In [6]:
txt_file_path = os.getenv("HOME")+'/content/drive/MyDrive/Exploration/Exploration_6/data/*'

txt_list = glob.glob(txt_file_path)
# txt_file_path 경로에 있는 모든 파일명을 리스트 형식으로 txt_list에 할당

raw_corpus=[]

# 여러개의 txt 파일을 모두 읽어서 raw_corpus에 담는다.
for txt_file in txt_list:
  with open(txt_file, "r") as f:
    raw = f.read().splitlines()  # read(): 파일 전체의 내용을 하나의 문자열로 읽어온다.
                                 # splitlines(): 여러 라인으로 구분되어 있는 문자열을 한라일씩 분리하여 리스트로 반환
    raw_corpus.extend(raw)   # expoend(): 리스트 함수로 추가적인 내용을 연장 한다.

print("데이터 크기:", len(raw_corpus))
print("Exmaples:\n" , raw_corpus[:3])

데이터 크기: 187088
Exmaples:
 ['Looking for some education', 'Made my way into the night', 'All that bullshit conversation']


# 데이터 정제

In [7]:
for idx, sentence in enumerate(raw_corpus):
  if len(sentence) == 0: continue  # 길이가 0인 문장은 건너 뛴다.
  
  if idx > 9: break
  print(sentence)

Looking for some education
Made my way into the night
All that bullshit conversation
Baby, can't you read the signs? I won't bore you with the details, baby
I don't even wanna waste your time
Let's just say that maybe
You could help me ease my mind
I ain't Mr. Right But if you're looking for fast love
If that's love in your eyes
It's more than enough


In [8]:
def preprocess_sentence(sentence):
  sentence = sentence.lower().strip()                   # 1. 소문자 & 양쪽 공백 지우기
  sentence = re.sub(r"([?.!,¿])", r" \1 ",sentence)     # 2. 특수문자 양쪽에 공백 넣기
  sentence = re.sub(r'[" "]+', " ", sentence)           # 3. 여러 공백을 하나의 공백으로 바꾸기
  sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4. a-zA-Z?.!,¿ 가 아닌 모든 문자를 하나의 공백으로 바꾼다.
  sentence = sentence.strip()                           # 5. 다시 양쪽 공백 지우기
  sentence = '<start> ' + sentence + ' <end>'           # 6. 문장 시작에는 <start>, 끝에는 <end>를 추가

  return sentence
print(preprocess_sentence("This @_is ;;;sample        sentence.")) 

<start> this is sample sentence . <end>


In [9]:
corpus =[]
for sentence in raw_corpus:
  if len(sentence) == 0: continue
  if sentence[-1] == ":": continue

  preprocessed_sentence = preprocess_sentence(sentence)
  corpus.append(preprocessed_sentence)

corpus[:10]

['<start> looking for some education <end>',
 '<start> made my way into the night <end>',
 '<start> all that bullshit conversation <end>',
 '<start> baby , can t you read the signs ? i won t bore you with the details , baby <end>',
 '<start> i don t even wanna waste your time <end>',
 '<start> let s just say that maybe <end>',
 '<start> you could help me ease my mind <end>',
 '<start> i ain t mr . right but if you re looking for fast love <end>',
 '<start> if that s love in your eyes <end>',
 '<start> it s more than enough <end>']

In [10]:
def tokenize(corpus):
  # 12,000 단어를 기억할 수 있는 tokenizer를 만든다.
  # 여기서는 이미 문장을 정리하여 fiolters가 필요없다.
  # 12,000 단어에 포함되지 못한 단어는 '<unk>'로 바꾼다.

  tokenizer = tf.keras.preprocessing.text.Tokenizer(
      num_words = 12000,
      filters = ' ',
      oov_token = '<unk>'
  )

  tokenizer.fit_on_texts(corpus)  # 문자 데이터를 입력받아 리스트의 형태로 변환하는 메서드
  tensor = tokenizer.texts_to_sequences(corpus)  # tokenizer를 이용해 corpus를 Tensor로 변환. 텍스트 안의 단어들을 숫자의 시퀀스 형태로 변환하는 메서드
  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding = 'post') # 입력 데이터의 시퀀스 길이를 일정하게 맞춘다. 패딩을 뒤쪽으로

  print(tensor, tokenizer)
  return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2 304  28 ...   0   0   0]
 [  2 221  13 ...   0   0   0]
 [  2  24  17 ...   0   0   0]
 ...
 [  2  23  77 ...   0   0   0]
 [  2  42  26 ...   0   0   0]
 [  2  23  77 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f7bf7a97a10>


In [11]:
for idx in tokenizer.index_word:
  print(idx, ":", tokenizer.index_word[idx])

  if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [12]:
# 제시 조건에 따라 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외

src_input = tensor[:,:15]  # tensor에서 마지막 토큰을 잘라서 소스 문장을 생성. 마지막 토큰은 <end>가 아니라 <pad>일 가능성 높다.

tgt_input = tensor[:,1:16]   # tensor에서 <start>를 잘라내서 타켓 문장을 생성

print(src_input[0])
print(tgt_input[0])

[   2  304   28   99 4811    3    0    0    0    0    0    0    0    0
    0]
[ 304   28   99 4811    3    0    0    0    0    0    0    0    0    0
    0]


# 평가 데이터셋 분리

In [13]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,tgt_input, test_size=0.2, random_state=1005)

# 인공지능 만들기

In [65]:
BUFFER_SIZE = len(enc_train)  # buffer_size 는 보통 전체 데이터의 크기와 같거나 조금 크게 설정
BATCH_SIZE = 128
steps_per_epoch = len(enc_train) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1

dataset = tf.data.Dataset.from_tensor_slices((enc_train,dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder = True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(128, 15), dtype=tf.int32, name=None), TensorSpec(shape=(128, 15), dtype=tf.int32, name=None))>

In [66]:
class TextGenerator(tf.keras.Model):
  def __init__(self, vocab_size, embedding_size, hidden_size):
    super().__init__()
        # Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이로 구성
    # Embedding 레이어는 단어 산전의 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔준다.
    # 이 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현으로 사용된다.
    # (EX. 만약 embedding_size 가 2라면)
    # 차갑다:[0.0, 1.0] / 뜨겁다: [1.0, 0.0], 미지근하다:[0.5, 0.5]

    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
    self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
    self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
    self.linear = tf.keras.layers.Dense(vocab_size)

  def call(self,x):
    out = self.embedding(x)
    out = self.rnn_1(out)
    out = self.rnn_2(out)
    out = self.linear(out)

    return out

# embedding size 값이 커질수록 단어의 추상적인 특징들을 더 잡아낼 수 있지만
# 그만큼 충분한 데이터가 없으면 안좋은 결과 값을 가져온다.

embedding_size = 512 # 워드 벡터의 차원수를 말하며 단어가 추상적으로 표현되는 크기이다.
hidden_size = 2048   # 모델에 얼마나 많은 일꾼을 둘 것인가? 정도로 이해..
lyricist = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size) 
# tokenizer.num_words 에 +1인 이유는 문장에 없는 pad 가 사용되었기 때문이다.

In [67]:
# 데이터셋에서 데이터 한 배치만 불러온다
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어둔다.
lyricist(src_sample)

<tf.Tensor: shape=(128, 15, 12001), dtype=float32, numpy=
array([[[-1.3629909e-04,  2.7123376e-04,  3.7818207e-04, ...,
          5.8038961e-05,  3.4258395e-04,  7.5319813e-06],
        [-3.5436207e-04,  3.9922434e-04,  5.6079961e-04, ...,
          1.3399466e-04,  4.8866437e-04,  3.6288882e-04],
        [-8.5920835e-04, -3.4527540e-05,  5.8212283e-04, ...,
          5.6013133e-04,  7.3939381e-04,  9.7106019e-04],
        ...,
        [ 5.6735007e-04, -6.8624545e-04, -9.0092374e-04, ...,
         -2.7201581e-04,  9.2132341e-06,  2.0301384e-03],
        [ 1.1467069e-03, -1.3380365e-03, -1.5325662e-03, ...,
         -1.0270918e-04,  2.6411720e-04,  2.0321782e-03],
        [ 1.6802745e-03, -1.9834512e-03, -2.1569976e-03, ...,
          9.6398646e-05,  5.7963218e-04,  2.0371519e-03]],

       [[-1.3629909e-04,  2.7123376e-04,  3.7818207e-04, ...,
          5.8038961e-05,  3.4258395e-04,  7.5319813e-06],
        [-4.3673474e-05,  4.0245560e-04,  6.1933714e-04, ...,
          1.5011622e-04, 

In [68]:
lyricist.summary()

Model: "text_generator_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     multiple                  6144512   
                                                                 
 lstm_16 (LSTM)              multiple                  20979712  
                                                                 
 lstm_17 (LSTM)              multiple                  33562624  
                                                                 
 dense_8 (Dense)             multiple                  24590049  
                                                                 
Total params: 85,276,897
Trainable params: 85,276,897
Non-trainable params: 0
_________________________________________________________________


In [69]:
optimizer = tf.keras.optimizers.Adam()  # Adam은 현재 가장 많이 사용하는 옵티마이저이다.
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits = True,
    reduction = 'none'
)

lyricist.compile(loss=loss, optimizer = optimizer)
lyricist.fit(dataset, validation_data=(enc_val, dec_val), epochs=6)

Epoch 1/6
1098/1098 [==============================] - 370s 335ms/step - loss: 3.1320 - val_loss: 2.8075
Epoch 2/6
1098/1098 [==============================] - 371s 338ms/step - loss: 2.5943 - val_loss: 2.5343
Epoch 3/6
1098/1098 [==============================] - 371s 338ms/step - loss: 2.2163 - val_loss: 2.3576
Epoch 4/6
1098/1098 [==============================] - 371s 338ms/step - loss: 1.8674 - val_loss: 2.2437
Epoch 5/6
1098/1098 [==============================] - 409s 373ms/step - loss: 1.5683 - val_loss: 2.1858
Epoch 6/6
1098/1098 [==============================] - 370s 337ms/step - loss: 1.3331 - val_loss: 2.1688


In [70]:
# 문장생성 함수 정의
# 모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행

def generate_text(lyricist,tokenizer, init_sentence="<start>", max_len = 20): # 시작 문다열을 init_sentence로 받으며 디폴트값은 <start>를 받는다.
  # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환
    test_input = tokenizer.texts_to_sequences([init_sentence]) #텍스트 안의 단어들을 숫자의 시퀀스의 형태로 변환
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

  # 단어 하나씩 예측해 문장을 만든다.
  #   1. 입력받은 문장의 텐서를 입력한다.
  #   2. 예측된 값 중 가장 높은 확률인 word index를 뽑아낸다.
  #   3. 2에서 예측된 word index를 문장 뒤에 붙인다.
  #   4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마친다.(도달하지 못하였으면 while 루프를 돌면서 다음 단어를 예측)
  
    while True:                # 루프를 돌면서 init_sentence에 단어를 하나씩 생성
        # 1
        predict = lyricist(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4 
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated #최종적으로 모델이 생성한 문장을 반환

In [71]:
generate_text(lyricist, tokenizer, init_sentence="<start> i love")  # 시작문장으로 i love 를 넣어 문장생성 함수 실행

'<start> i love you so much <end> '